In [1]:
!pip install langchain langchain-core langchain-community pydantic langchain_experimental

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires reque

In [2]:
!pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 115.1 MB/s eta 0:00:00


In [3]:
!pip install -q langchain_huggingface langchain-core requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 33.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.3.4 requires langchain-core<0.4.0,>=0.3.28, but you have langchain-core 1.0.5 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.5 which is incompatible.


# 1) Tool Creation

In [4]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace, HuggingFaceEmbeddings
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [ ]:
# Tool 1
@tool
def getConversion(base_currency: str, target_currency: str) -> float:
  """
  This Function hits the API and fetches the currency conversion factor between a given base currency and a target currency
  """
  # "https://v6.exchangerate-api.com/v6/YOUR-API-KEY/pair/EUR/GBP"

  url = f"https://v6.exchangerate-api.com/v6//pair/{base_currency}/{target_currency}"

  response = requests.get(url)
  return response.json()


# Tool 2
@tool
def convert(baseCurrencyValue: float, conversionFactor: float) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return baseCurrencyValue * conversionFactor

In [ ]:
getConversion.invoke({'base_currency':"USD", 'target_currency': 'INR'})

In [12]:
convert.invoke({'baseCurrencyValue': 10, 'conversionFactor':88.7636})

887.636

# 2) Tool Binding

In [44]:
llm = HuggingFaceEndpoint(
    repo_id = "Qwen/Qwen3-4B-Instruct-2507",
    task = "text-generation",
)
model = ChatHuggingFace(llm = llm)

In [45]:
llmWithTools = model.bind_tools([getConversion, convert])

In [46]:
messages = [
    HumanMessage("What is the conversion factor between USD and INR, and based on that can you convert 100 USD to INR?")
]

In [48]:
aiMessage = llmWithTools.invoke(messages)

In [51]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage("What is the conversion factor between USD and INR, and based on that convert 100 USD to INR")
]


msg1 = llmWithTools.invoke(messages)
messages.append(msg1)

tool_call_1 = msg1.tool_calls[0]
tool_args_1 = tool_call_1["args"]

raw_tool_output_1 = getConversion.invoke(tool_args_1)

tool_output_1 = ToolMessage(
    name="getConversion",
    content=str(raw_tool_output_1),
    tool_call_id=tool_call_1["id"]
)

messages.append(tool_output_1)

msg2 = llmWithTools.invoke(messages)
messages.append(msg2)

tool_call_2 = msg2.tool_calls[0]
tool_args_2 = tool_call_2["args"]


raw_tool_output_2 = convert.invoke(tool_args_2)

tool_output_2 = ToolMessage(
    name="convert",
    content=str(raw_tool_output_2),
    tool_call_id=tool_call_2["id"]
)

messages.append(tool_output_2)

final_reply = llmWithTools.invoke(messages)
print(final_reply.content)


The conversion factor between USD and INR is 88.7636. 

Converting 100 USD to INR gives: **8,876.36 INR**.
